#### This lab used to discover my current rent property information in Chelsea Manhattan new York. helping me to explore facilities around this neighborhood. Because I am very satisfy with my location. so I want to live in a similar  neighborhood when I move in Toronto. 

In [22]:
import numpy as np #library to handle data in a vectorized manner 
import pandas as pd #library for data analysis 
pd.set_option('display.max_columns',None)
pd.set_option('display.max.rows',None)
import requests
import json #library to handle JSON files
import os
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
print(os.getcwd())

C:\Users\liuyi\Desktop\Coursera Capstone


In [4]:
with open('newyork_data.json') as json_file:
    newyork_data = json.load(json_file)

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude


In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#### This is the Neighorhood i live. Manhattan Chelsea.

In [16]:
manhattan_data=neighborhoods[(neighborhoods['Neighborhood']=='Chelsea')&(neighborhoods['Borough']=='Manhattan')]
manhattan_data

,Borough,Neighborhood,Latitude,Longitude
116,Manhattan,Chelsea,40.744035,-74.003116


Let's get the geographical coordinates of Manhattan.

In [11]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [14]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

#### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'DV1ORBRI554RS4C1DVX1EXJ2HBF1MGZ45AM5TMUUO4K2VIQC' # your Foursquare ID
CLIENT_SECRET = 'VK0JZOEDLNZTCQCRS124ZS1XII1GJ05DFEZKKYFSSVPW0UNX' # your Foursquare Secret
VERSION = '20200410' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DV1ORBRI554RS4C1DVX1EXJ2HBF1MGZ45AM5TMUUO4K2VIQC
CLIENT_SECRET:VK0JZOEDLNZTCQCRS124ZS1XII1GJ05DFEZKKYFSSVPW0UNX


#### Let's explore the Chelsea in our dataframe.

In [23]:
radius = 500
LIMIT = 100

venues = []

for lat, long,borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
             
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [39]:
### change to a dataframe 
selected_table=pd.DataFrame(venues)
selected_table.head()

,0,1,2,3,4,5,6,7
0,Manhattan,Chelsea,40.744035,-74.003116,Milk & Hops Chelsea,40.744751,-74.002595,Beer Bar
1,Manhattan,Chelsea,40.744035,-74.003116,Bathtub Gin,40.743638,-74.003290,Speakeasy
2,Manhattan,Chelsea,40.744035,-74.003116,Billy's Bakery,40.745374,-74.002119,Cupcake Shop
3,Manhattan,Chelsea,40.744035,-74.003116,Buddakan,40.742120,-74.004612,Chinese Restaurant
4,Manhattan,Chelsea,40.744035,-74.003116,The Joyce Theater,40.742829,-74.000534,Theater


In [41]:
#define column name for select_table
selected_table.columns =['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
selected_table

,Borough,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Manhattan,Chelsea,40.744035,-74.003116,Milk & Hops Chelsea,40.744751,-74.002595,Beer Bar
1,Manhattan,Chelsea,40.744035,-74.003116,Bathtub Gin,40.743638,-74.003290,Speakeasy
2,Manhattan,Chelsea,40.744035,-74.003116,Billy's Bakery,40.745374,-74.002119,Cupcake Shop
3,Manhattan,Chelsea,40.744035,-74.003116,Buddakan,40.742120,-74.004612,Chinese Restaurant
4,Manhattan,Chelsea,40.744035,-74.003116,The Joyce Theater,40.742829,-74.000534,Theater
5,Manhattan,Chelsea,40.744035,-74.003116,Chelsea Market,40.742182,-74.005301,Market
6,Manhattan,Chelsea,40.744035,-74.003116,Sushinao,40.742033,-74.001581,Japanese Restaurant
7,Manhattan,Chelsea,40.744035,-74.003116,The Meatball Shop,40.745988,-74.001686,Italian Restaurant
8,Manhattan,Chelsea,40.744035,-74.003116,Intelligentsia Coffee,40.745926,-74.005376,Coffee Shop
9,Manhattan,Chelsea,40.744035,-74.003116,Lobster Place,40.742715,-74.006107,Fish Market


### Analyze Chelase Neighborhood

In [53]:
# one hot encoding
Chelsea = pd.get_dummies(selected_table[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Chelsea ['Neighborhood'] =selected_table['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = [Chelsea.columns[-1]] + list(Chelsea.columns[:-1])
Chelsea  = Chelsea[fixed_columns]


Chelsea.head()


,Neighborhood,American Restaurant,Art Gallery,Bagel Shop,Bakery,Bar,Beer Bar,Bookstore,Boutique,Burger Joint,Butcher,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cupcake Shop,Cycle Studio,Fish Market,Flea Market,French Restaurant,General Entertainment,Gift Shop,Grocery Store,Gym,Harbor / Marina,Health & Beauty Service,Hotel,Ice Cream Shop,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Nightclub,Office,Park,Pet Store,Photography Studio,Physical Therapist,Pizza Place,Plaza,Pool,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Spanish Restaurant,Speakeasy,Taco Place,Theater,Wine Bar,Wine Shop,Women's Store
0,Chelsea,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Chelsea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Chelsea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Chelsea,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Chelsea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [55]:
Chelsea_freq=Chelsea.groupby('Neighborhood').mean().reset_index()
Chelsea_freq.head()

,Neighborhood,American Restaurant,Art Gallery,Bagel Shop,Bakery,Bar,Beer Bar,Bookstore,Boutique,Burger Joint,Butcher,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cupcake Shop,Cycle Studio,Fish Market,Flea Market,French Restaurant,General Entertainment,Gift Shop,Grocery Store,Gym,Harbor / Marina,Health & Beauty Service,Hotel,Ice Cream Shop,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Nightclub,Office,Park,Pet Store,Photography Studio,Physical Therapist,Pizza Place,Plaza,Pool,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Spanish Restaurant,Speakeasy,Taco Place,Theater,Wine Bar,Wine Shop,Women's Store
0,Chelsea,0.03,0.12,0.01,0.02,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.09,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


### Let's find top 10 most common venues


In [57]:
num_top_venues = 10

for hood in Chelsea_freq['Neighborhood']:
    print("----"+hood+"----")
    temp =Chelsea_freq[Chelsea_freq['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Chelsea----
                 venue  freq
0          Art Gallery  0.12
1          Coffee Shop  0.09
2  American Restaurant  0.03
3   Seafood Restaurant  0.03
4       Ice Cream Shop  0.03
5   Italian Restaurant  0.03
6            Bookstore  0.02
7               Market  0.02
8             Boutique  0.02
9                 Café  0.02




In [59]:
# we can use write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Chelsea_freq['Neighborhood']

for ind in np.arange(Chelsea_freq.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Chelsea_freq.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chelsea,Art Gallery,Coffee Shop,American Restaurant,Ice Cream Shop,Italian Restaurant,Seafood Restaurant,Cupcake Shop,Park,Pizza Place,Market


So based on table above , i's like to find a same or similar neighborhood in toronto, then thit neighborhood will be my dream place. Generally speaking, i did't put rental pricing as condition, becasue in my opinion new york rental price is higher than in toronto. So it should be cheaper of same neighorhood in toronto.